# 打车软件

---

## 概述

数据来源：

<https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page>


数据描述：

<https://www1.nyc.gov/assets/tlc/downloads/pdf/data_dictionary_trip_records_yellow.pdf>

导入相关包

In [1]:
import os
import time

from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import seaborn as sns

观察数据

In [2]:
data = pd.read_csv("data/yellow_tripdata_2020-06.csv")
data.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,1.0,2020-06-01 00:31:23,2020-06-01 00:49:58,1.0,3.6,1.0,N,140,68,1.0,15.5,3.0,0.5,4.00,0.0,0.3,23.30,2.5
1,1.0,2020-06-01 00:42:50,2020-06-01 01:04:33,1.0,5.6,1.0,N,79,226,1.0,19.5,3.0,0.5,2.00,0.0,0.3,25.30,2.5
2,1.0,2020-06-01 00:39:51,2020-06-01 00:49:09,1.0,2.3,1.0,N,238,116,2.0,10.0,0.5,0.5,0.00,0.0,0.3,11.30,0.0
3,1.0,2020-06-01 00:56:13,2020-06-01 01:11:38,1.0,5.3,1.0,N,141,116,2.0,17.5,3.0,0.5,0.00,0.0,0.3,21.30,2.5
4,1.0,2020-06-01 00:16:41,2020-06-01 00:29:30,1.0,4.4,1.0,N,186,75,1.0,14.5,3.0,0.5,3.65,0.0,0.3,21.95,2.5


profiling分析

In [3]:
import pandas_profiling
pfr = pandas_profiling.ProfileReport(data)
pfr.to_file('report.html')

## 数据分析

### 变量分析

对profiling得到的报告进行分析

|变量|含义|类型|待处理|
|:-|:-|:-|:-|
|VendorID|记录提供商|Categorical|缺省9.2%|
|tpep_pickup_datetime|上车时间|Datetime||
|tpep_dropoff_datetime|下车时间|Datetime||
|passenger_count|乘客数量|Categorical Number|缺省9.2%，零值2.4%|
|trip_distance|打车距离|Contiguous Number|零值3.2%，存在异常值(极大值)|
|RatecodeID|价格代码|Categorical Number|缺省9.2%|
|store_and_fwd_flag|是否保存在车辆内存中|Boolean|缺省9.2%|
|PULocationID|上车区域ID|Categorical Number||
|DOLocationID|下车区域ID|Categorical Number||
|payment_type|支付类别|Categorical Number|缺省9.2%|
|fare_amount|距离计价|Contiguous Number|存在异常值(负值，极大值)|
|extra|附加费|Contiguous Number|零值47.8%(正常)|
|mta_tax|交通税|Categorical Number|两类|
|tip_amount|小费|Contiguous Number|零值41.0%(正常)|
|tolls_amount|过路费|Contiguous Number|零值94.7%(正常)，存在异常值(极大值)|
|improvement_surcharge|改善附加费|Categorical Number|两类，存在异常值(负值)|
|total_amount|总费用|Contiguous Number|存在异常值(负值，极大值)|
|congestion_surcharge|拥挤附加费|Categorical Number|两类，存在异常值(负值)|

### 上下车差值

有1-265个区域，我们以一个小时为观测时间，该段时间内该区域下车记录减去上车记录，存在正负值（负值表示上车的车辆数更多）

In [4]:
time_format ="%Y-%m-%d %H:%M:%S"

def cal_diff(filename, write = True):
    data = pd.read_csv(filename).drop(columns=['Unnamed: 0'])
    year_month = filename.split("_")[-1].split(".")[0]
    
    start_time = datetime.strptime(year_month, "%Y-%m")
    end_time = start_time + relativedelta(months = 1)

    # start_time = "2020-6-1 00:00:00"
    # end_time = "2020-6-2 00:00:00"

    # start_time = datetime.strptime(start_time, time_format)
    # end_time = datetime.strptime(end_time, time_format)

    data["tpep_pickup_datetime"] = pd.to_datetime(data["tpep_pickup_datetime"])
    data["tpep_dropoff_datetime"] = pd.to_datetime(data["tpep_dropoff_datetime"])

    res_times = []
    res_datas = []

    print("Calculating file %s ..." % filename)

    while start_time < end_time:
        # print(start_time)
        # res_times.append(datetime.strftime(start_time, time_format))
        res_times.append(start_time)
        
        tmp = start_time + timedelta(hours = 1)

        # 对每个小时进行统计
        res_data = np.zeros(265, dtype=np.int)

        pickup = data.loc[(data["tpep_pickup_datetime"] > start_time) & (data["tpep_pickup_datetime"] < tmp)]["PULocationID"]
        pickup.reset_index(drop=True, inplace=True)
        dropoff = data.loc[(data["tpep_dropoff_datetime"] > start_time) & (data["tpep_dropoff_datetime"] < tmp)]["DOLocationID"]
        dropoff.reset_index(drop=True, inplace=True)
        for i in range(pickup.shape[0]):
            res_data[pickup[i] - 1] -= 1
        for i in range(dropoff.shape[0]):
            res_data[dropoff[i] - 1] += 1
        
        res_datas.append(res_data)

        start_time = tmp

    res_times = pd.DataFrame(np.array(res_times)).rename(columns = {0: 'time'})
    res_datas = pd.DataFrame(np.array(res_datas))

    res = pd.concat([res_times, res_datas], axis = 1)
    if write:
        res.to_csv("data/diff/%s.csv" % (year_month), index=False)
    print("Done!")
    return res

# cal_diff("data/clear_data/new_yellow_tripdata_2020-06.csv")

我们对数据清洗过的一年半的数据进行差值计算

In [18]:
for root, dirs, files in os.walk("data/clear_data"):
    for f in files:
        filename = os.path.join(root, f)
        start = time.time()
        cal_diff(filename)
        print("Cost time: %.2fs" % (time.time() - start))
        print("--------" * 10)

Calculating file data/clear_data\new_yellow_tripdata_2019-01.csv ...
Done!
Cost time: 232.47
--------------------------------------------------------------------------------
Calculating file data/clear_data\new_yellow_tripdata_2019-02.csv ...
Done!
Cost time: 210.01
--------------------------------------------------------------------------------
Calculating file data/clear_data\new_yellow_tripdata_2019-03.csv ...
Done!
Cost time: 240.43
--------------------------------------------------------------------------------
Calculating file data/clear_data\new_yellow_tripdata_2019-04.csv ...
Done!
Cost time: 225.02
--------------------------------------------------------------------------------
Calculating file data/clear_data\new_yellow_tripdata_2019-05.csv ...
Done!
Cost time: 230.66
--------------------------------------------------------------------------------
Calculating file data/clear_data\new_yellow_tripdata_2019-06.csv ...
Done!
Cost time: 209.85
-------------------------------------

`matplotlib画图`

In [2]:
filename = "data/diff/2020-06.csv" # diff文件名
frames = 100 # 展示帧数
pause = 0.1 # 暂停时间，单位s

In [3]:
# res = cal_diff("data/clear_data/new_yellow_tripdata_2020-06.csv", write=False)
res = pd.read_csv(filename)
res.head()

,time,0,1,2,3,4,5,6,7,8,...,255,256,257,258,259,260,261,262,263,264
0,2020-06-01 00:00:00,0,0,0,0,0,0,-1,0,0,...,1,0,1,0,0,0,0,3,0,1
1,2020-06-01 01:00:00,0,0,0,0,0,0,2,0,0,...,0,0,-2,1,-1,-1,1,0,-1,0
2,2020-06-01 02:00:00,0,0,0,0,0,0,3,0,0,...,0,0,1,0,1,0,0,1,1,1
3,2020-06-01 03:00:00,0,0,0,0,0,0,0,0,0,...,-1,0,-3,-1,0,2,-3,-3,0,0
4,2020-06-01 04:00:00,0,0,0,0,0,0,-2,0,1,...,0,0,0,-2,0,-1,2,-3,0,0


In [5]:
%matplotlib qt

res_datas = res.iloc[:, 1:]
res_times = res.iloc[:, 0]

# print(res_times)
# print(res_datas.index)

# 创建画布
sns.set_style("whitegrid")
fig = plt.figure(figsize=(15, 10))

plt.ion()

plt.rcParams['font.sans-serif'] = ['SimHei']  # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号

for i in range(res.shape[0]):

    if i == frames:
        break

    plt_data = res_datas.loc[i]
    plt_data.index = plt_data.index.astype(int)
    plt_data0 = plt_data.loc[(plt_data > 0) & (plt_data <= 20)]
    plt_data1 = plt_data.loc[(plt_data < 0) & (plt_data >= -20)]
    plt_data2 = plt_data.loc[plt_data > 20]
    plt_data3 = plt_data.loc[plt_data < -20]

    plt.cla()

    plt.xlabel('区域ID', fontsize=20)
    plt.ylabel('下车-上车', fontsize=20)
    plt.xlim([0, 264])
    plt.ylim([-20, 20])

    plt.title(res_times[i], fontsize=30)

    bar0 = plt.bar(x=plt_data0.index.values, height=plt_data0, color='r')
    bar1 = plt.bar(x=plt_data1.index.values, height=plt_data1, color='b')
    bar2 = plt.bar(x=plt_data2.index.values, height=plt_data2, color='orange')
    bar3 = plt.bar(x=plt_data3.index.values, height=plt_data3, color='purple')
    # print(plt_data3.index.values)
    for a, b in zip(plt_data2.index.values, plt_data2):
        text0 = plt.text(a - 6, 18.5, '%.0f'%b, ha = 'center', va = 'bottom', fontsize=20, color='orange')
    for a, b in zip(plt_data3.index.values, plt_data3):
        text1 = plt.text(a - 6, -20, '%.0f'%b, ha = 'center', va = 'bottom', fontsize=20, color='purple')

    plt.pause(pause)

plt.ioff()

plt.show()

`pyecharts画图`

### 区域间旅程距离

计算一个265x265的数组表示各个区域间平均旅程距离

In [39]:
filename = "data/clear_data/new_yellow_tripdata_2020-06.csv"
data = pd.read_csv(filename).drop(columns=['Unnamed: 0'])[['trip_distance', "PULocationID", "DOLocationID"]]
data.head()

,trip_distance,PULocationID,DOLocationID
0,3.6,140,68
1,5.6,79,226
2,2.3,238,116
3,5.3,141,116
4,4.4,186,75


统计一个月记录下的平均距离

In [42]:
def cal_dist(filename, write = True):
    year_month = filename.split("_")[-1].split(".")[0]

    data = pd.read_csv(filename).drop(columns=['Unnamed: 0'])[['trip_distance', "PULocationID", "DOLocationID"]]

    dist = np.zeros((265, 265))
    cnt = np.zeros((265, 265), dtype=np.int)

    print("Calculating file %s ..." % filename)

    start = time.time()
    for i in range(data.shape[0]):

        if (i + 1) % 200000 == 0:
            print("%d: cost time %.2fs" %(i + 1, time.time() - start))
            start = time.time()

        tmp = data.loc[i]
        d = tmp["trip_distance"]
        id1 = int(tmp["PULocationID"]) - 1
        id2 = int(tmp["DOLocationID"]) - 1
        
        dist[id1, id2] = dist[id1, id2] + d
        dist[id2, id1] = dist[id1, id2]
        cnt[id1, id2] = cnt[id1, id2] + 1
        cnt[id2, id1] = cnt[id1, id2]

    cnt[np.where(cnt == 0)] = 1
    res = dist / cnt

    res = pd.DataFrame(res)
    if write:
        res.to_csv("data/dist/%s.csv" % (year_month), index=False)
    print("Done!")
    
    return res

计算一个月740w条数据约20分钟

In [ ]:
i = 0
for root, dirs, files in os.walk("data/clear_data"):
    for f in files:
        if i == 2: # 只读取几个文件
            break
        i += 1
        filename = os.path.join(root, f)
        start = time.time()
        cal_dist(filename)
        print("Cost time: %.2fs" % (time.time() - start))
        print("--------" * 10)

## 模型搭建

通过观察分析，对我们挑选出的一些数据（由于全数据集较大）进行模型搭建：

区域ID：220-264

`模型`

一个图表示

* 节点：上下车的差值
  * 正，表示下车的出租数量更多（假设为空车数）
  * 负，表示上车的出租数量更多（假设为需求数）
* 边：出租车建议去下个区域的推荐值
  * 主要由我们统计得到的区域间距离及该时间段该区域的上下车差值估计的推荐值

### 数据获取

区域间trip距离的估计我们认为随时间变化不大，所以仅选择了2019-01月作为代表

In [61]:
data = pd.read_csv("data/dist/2019-01.csv").iloc[220:265, 220:265]
data.to_csv("data/model/dist.csv", index = False)

同理，2019-01，月平均每一个小时上下车差值

In [74]:
data = pd.read_csv("data/diff/2019-01.csv").iloc[:, 221:]
days = data.shape[0] // 24
mean = data.iloc[0:24].values
for i in range(1,days):
    tmp = data.iloc[i*24: (i + 1) * 24].values
    mean = mean + tmp
mean = mean / days
mean = pd.DataFrame(mean)
mean.columns = range(220,265)
mean.to_csv("data/model/diff.csv", index = False)

### 边的权重

推荐值

$
weight_{i, j} = 
\begin{cases}
\left|{\frac{diff_i \times diff_j}{dist_{i,j}}}\right|, & if\quad diff_i > 0\quad and\quad diff_j < 0 \\
0, & otherwise
\end{cases}
$

$weight_{i, j}$表示有向边i -> j的权重，$diff_i$表示节点i的值（空车数），$diff_j$表示节点j的值（需求数），$dist_{i, j}$表示节点i到j的旅途距离

In [77]:
dist = pd.read_csv("data/model/dist.csv")
diff = pd.read_csv("data/model/diff.csv")

In [94]:
dist = np.array(dist)
dist[np.where(dist == 0)] = 10000
dist = pd.DataFrame(dist)

In [95]:
res = []
# start = time.time()
for i in range(24):
    weights = np.zeros((45, 45))
    for j in range(45):
        for k in range(45):
            diff_i = diff.iloc[i, j]
            diff_j = diff.iloc[i, k]
            if diff_i > 0 and diff_j < 0:
                weights[i, j] = abs(diff_i * diff_j / dist.iloc[i, j])
                # print(diff_i, diff_j, dist.iloc[i, j], weights[i, j])
            else:
                continue
    res.append(weights)
    # print("%d : cost time %.2f" % (i, (time.time() - start)))
    # start = time.time()

876760855
1.4516129032258065 -6.290322580645161 7.07 1.2915294799882842
0.3548387096774194 -442.29032258064524 10000.0 0.015694172736732574
0.3548387096774194 -18.41935483870968 10000.0 0.0006535900104058274
0.3548387096774194 -207.90322580645162 10000.0 0.007377211238293446
0.3548387096774194 -40.25806451612903 10000.0 0.0014285119667013528
0.3548387096774194 -87.09677419354837 10000.0 0.0030905306971904262
0.3548387096774194 -6.290322580645161 10000.0 0.00022320499479708635
2.290322580645161 -442.29032258064524 10000.0 0.10129875130072842
2.290322580645161 -18.41935483870968 10000.0 0.004218626430801249
2.290322580645161 -207.90322580645162 10000.0 0.04761654526534859
2.290322580645161 -40.25806451612903 10000.0 0.009220395421436002
2.290322580645161 -87.09677419354837 10000.0 0.019947970863683655
2.290322580645161 -6.290322580645161 10000.0 0.0014406867845993755
0.03225806451612903 -442.29032258064524 10000.0 0.0014267429760665975
0.03225806451612903 -18.41935483870968 10000.0 5.941

In [84]:
res

[array([[35.29842634,         inf,         inf, ...,         inf,
                 inf, 66.39930751],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        ...,
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ]]),
 array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [        inf, 30.34327847,         inf, ...,         inf,
         36.28582686, 25.50716839],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        ...,
        [ 0.        ,  0.        ,  0.        , ...,  

## 附录

使用下列命令查看matplotlib使用的后端

In [7]:
%pylab

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib
